In [2]:

import psycopg2
import yfinance as yf
import logging
import pandas as pd
from datetime import datetime, date, timedelta
#from datetime import date, timedelta
# Database connection parameters
DB_PARAMS = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "Aarav12#$",
    "host": "npatelnynj.ddns.net",
    "port": "5432"
}

# Generate timestamped log file
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_filename = f"daily_stock_log_{timestamp}.txt"

# Configure logging
logging.basicConfig(
    filename=log_filename,
    level=logging.INFO,
    format="%(asctime)s: %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S"
)

logging.info("************ Daily Stock Data Insertion Script Started ************")

# Function to fetch tickers from the database
def get_tickers():
    try:
        conn = psycopg2.connect(**DB_PARAMS)
        cur = conn.cursor()
        cur.execute("SELECT ticker FROM companies;")
        tickers = cur.fetchall()  # List of (ticker,)
        conn.close()
        logging.info(f"Fetched {len(tickers)} tickers from database.")
        return [ticker[0] for ticker in tickers]  # Extract ticker strings
    except Exception as e:
        logging.error(f"Failed to fetch tickers: {e}")
        return []

# Function to fetch daily stock data from Yahoo Finance
def fetch_daily_stock_data(ticker):
    try:
        # Set fetch_date to today’s date automatically
        fetch_date = date.today().strftime("%Y-%m-%d")
        fetch_date_dt = datetime.strptime(fetch_date, "%Y-%m-%d")
        start_date = fetch_date_dt.strftime("%Y-%m-%d")
        # Set end date to the next day to ensure we capture today
        end_date = (fetch_date_dt + timedelta(days=1)).strftime("%Y-%m-%d")

        print(f"Fetching data for {ticker} on {start_date}...")
        stock_data = yf.download(ticker, start=start_date, end=end_date, progress=False)

        if stock_data.empty:
            logging.warning(f"No data found for {ticker} on {start_date}")
            print(f"⚠️ No data found for {ticker} on {start_date}. Skipping.")
            return None

        # Reset index to move 'Date' from index to a column
        stock_data.reset_index(inplace=True)

        # Filter for the exact date we want (today)
        stock_data = stock_data[stock_data['Date'].dt.strftime("%Y-%m-%d") == fetch_date]

        if stock_data.empty:
            logging.warning(f"No data found for {ticker} on {fetch_date}")
            print(f"⚠️ No data found for {ticker} on {fetch_date}. Skipping.")
            return None

        # Flatten MultiIndex columns
        stock_data.columns = [col[0] if col[0] == 'Date' else col[0] for col in stock_data.columns]

        # Convert DataFrame to a list of tuples for insertion
        result = [
            (
                ticker,
                row['Date'],
                float(row['Open']) if pd.notna(row['Open']) else None,
                float(row['High']) if pd.notna(row['High']) else None,
                float(row['Low']) if pd.notna(row['Low']) else None,
                float(row['Close']) if pd.notna(row['Close']) else None,
                int(row['Volume']) if pd.notna(row['Volume']) else None
            )
            for _, row in stock_data.iterrows()
        ]
        
        logging.info(f"Fetched 1 row for {ticker} on {fetch_date}.")
        return result
    except Exception as e:
        logging.error(f"Failed to fetch data for {ticker}: {e}")
        return None

# Function to insert daily stock data into the database
def insert_daily_stock_data(ticker, stock_data):
    if not stock_data:
        logging.error(f"No valid data to insert for {ticker}. Skipping.")
        return 0

    try:
        conn = psycopg2.connect(**DB_PARAMS)
        cur = conn.cursor()

        insert_query = """
        INSERT INTO stockdatadaily (stock_name, trade_date, open_price, high_price, low_price, close_price, volume)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (stock_name, trade_date) DO NOTHING;
        """

        cur.executemany(insert_query, stock_data)
        conn.commit()

        inserted_count = cur.rowcount
        cur.close()
        conn.close()
        
        logging.info(f"Inserted {inserted_count} rows for {ticker}.")
        return inserted_count
    except Exception as e:
        logging.error(f"Failed to insert data for {ticker}: {e}")
        return 0

# Main function to process daily stock data
def process_daily_spy_stocks():
    tickers = get_tickers()
    if not tickers:
        logging.error("No tickers found. Exiting process.")
        print("❌ No tickers found. Exiting.")
        return

    #fetch_date = date.today().strftime("%Y-%m-%d")  # Current day only
    fetch_date = (date.today() - timedelta(days=1)).strftime("%Y-%m-%d")
    
    logging.info("****************************************************************")
    logging.info(f"Daily Stock Data Insertion Started for {fetch_date}")
    logging.info("****************************************************************")
    logging.info(f"Total Number of Tickers: {len(tickers)}")
    logging.info("****************************************************************")

    print(f"Starting daily stock data insertion for {fetch_date}...")

    for idx, ticker in enumerate(tickers, start=1):
        start_time = datetime.now().strftime("%H:%M:%S")

        print(f"\n[{start_time}] Processing ({idx}/{len(tickers)}) - {ticker}...")
        logging.info(f"\nProcessing {ticker} (Ticker {idx}/{len(tickers)})")
        logging.info(f"Fetching data for {fetch_date}")

        stock_data = fetch_daily_stock_data(ticker)
        if not stock_data:
            logging.error(f"Skipping {ticker} due to missing or empty data.")
            continue

        print(f"📊 {ticker}: 1 row fetched. Inserting into DB...")
        inserted_count = insert_daily_stock_data(ticker, stock_data)

        if inserted_count > 0:
            logging.info(f"✅ {ticker}: Inserted {inserted_count} row(s).")
            print(f"✅ {ticker}: Inserted {inserted_count} row(s).")
        else:
            logging.error(f"❌ {ticker}: Failed to insert.")
            print(f"❌ {ticker}: Failed to insert.")

        logging.info("****************************************************************")

    print(f"\nDaily stock data insertion completed for {fetch_date}.")
    logging.info("************ Daily Stock Data Insertion Script Completed ************")

# Run the script
process_daily_spy_stocks()

Starting daily stock data insertion for 2025-04-03...

[21:24:08] Processing (1/503) - MMM...
Fetching data for MMM on 2025-04-04...
YF.download() has changed argument auto_adjust default to True
📊 MMM: 1 row fetched. Inserting into DB...
✅ MMM: Inserted 1 row(s).

[21:24:09] Processing (2/503) - AOS...
Fetching data for AOS on 2025-04-04...
📊 AOS: 1 row fetched. Inserting into DB...
✅ AOS: Inserted 1 row(s).

[21:24:09] Processing (3/503) - ABT...
Fetching data for ABT on 2025-04-04...
📊 ABT: 1 row fetched. Inserting into DB...
✅ ABT: Inserted 1 row(s).

[21:24:10] Processing (4/503) - ABBV...
Fetching data for ABBV on 2025-04-04...
📊 ABBV: 1 row fetched. Inserting into DB...
✅ ABBV: Inserted 1 row(s).

[21:24:10] Processing (5/503) - ACN...
Fetching data for ACN on 2025-04-04...
📊 ACN: 1 row fetched. Inserting into DB...
✅ ACN: Inserted 1 row(s).

[21:24:10] Processing (6/503) - ADBE...
Fetching data for ADBE on 2025-04-04...
📊 ADBE: 1 row fetched. Inserting into DB...
✅ ADBE: Insert